# Predicción de Cancer de mama mediante Random Forest, PCA and SVM


## Clase 1
Exponer explicación del trabajo

Dataset: https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+(Diagnostic)

La idea de la primera clase es aprender a realizar una exploración exhaustiva de los datos. Posteriormente se aprenderá a aplicar técnicas de selección de características. De evaluará el/los modelos aplicando y sin aplicar técnicas de Feature Selection y compararemos resultados.

## Práctica Machine Learning.

La idea es que sigan todos los pasos necesarios para crear un flujo de trabajo para utilizar algoritmos de Machine Learning:

1º Lo primero será la lectura y exploración tal y cómo se hizo en clase. Además, inetntar investigar nuevas técnicas/métodos de visualización de datos para extraer información para su posterior análisis (la exploración  de los datos debe de estar correctamente comentada y fundamentada)

2º Menejo de técnicas de pre-procesamiento de datos:
 - Evitar valores missing en el dataset
 - Estandarización/Normalización de los datos siempre y cuando sea necesario (se puede comparar los resultados aplicando la estandarización de los datos y sin aplicar)
 - Feature Selection. Aplicar algún/os método/s de selección de variables. La idea es entrenar el modelo con todas las características del dataset y evaluarlo, y después aplicar un método de selección de variables y volver a entrenar el modelo con las características seleccionadas y comparar resultados. Si se aplica más de un método de selección de variables se pueden comparar los resultados.
NOTA: para fijar el número de características elegidas por el método habrá que justificar el porqué, es decir, habrá que escoger el número óptimo de características para elegir el mejor modelo.
 - Dominio del Método PCA (Principal Component Analysis). Esta vez aplicaremos un PCA sobre el dataset completo y compararemos los resultados obtenidos.

3º Selección del modelo:
- Debemos de evaluar distintos modelos de clasificación y comparar sus resultados

4º Manejo de técnicas de hiperparametrización. Tenemos que saber aplicar al menos parametrización manual e hiperparametrización automática con el método GridSearch. Para definir el espacio de búsqueda de parámetros primero debemos de hacer un estudio previo del funcionamiento del algoritmo y de sus parámetros de entrada.

5º Evaluación del modelo. Debemos de conocer y saber aplicar distintas métricas para evaluar los modelos.

5.1º Además, se aplicará Cross-Validación para evaluar el modelo utilizando distintos fragmentos para entrenar y evaluar el modelo.


## Preparar entorno de trabajo

### Crear entorno virtual (Conda/Python env)

Una vez creado activamos el entorno con 'conda activate myentorno' para instalar la librerías necesarias

### Instalamos las librerías necesarias

Para instalar las librerías podemos hacerlo mediante "pip install" o "conda install"

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
%matplotlib inline

ModuleNotFoundError: No module named 'seaborn'

## Paso 1: Adquisición de los datos

In [ ]:
from sklearn.datasets import load_breast_cancer
breast_cancer = load_breast_cancer()

In [ ]:
# Observación de todos los elementos del dataset
# display(breast_cancer)

In [ ]:
# Elementos del dataset
breast_cancer.keys()

In [ ]:
# print(breast_cancer.DESCR[27:3130])

In [ ]:
# Observación de variables target
# print(breast_cancer.data)
# print(breast_cancer.target)
print(breast_cancer.target_names)

In [ ]:
print(breast_cancer.feature_names)

In [ ]:
print(breast_cancer.filename)
print(breast_cancer.data_module)

### Lectura de los datos
Crearemos un dataframe y lo rellenaremos con cada una de las "features" del dataset

In [ ]:
# cargamos los datos en un dataframe 
df_features = pd.DataFrame(breast_cancer.data, columns = breast_cancer.feature_names)
# Obtenemos información general del dataset
# df_features.info()

In [ ]:
df_features.head()

### Lectura de la variable "target"

In [ ]:
# 0 - Benigno
# 1 - Maligno
df_target = pd.DataFrame(breast_cancer.target, columns=['target'])
# df_target.head()

In [ ]:
df_target['target'].value_counts()

Según la descripción del conjunto de datos, la distribución de la variable objetivo es: 212 - Maligno, 357 - Benigno. Por lo tanto, "benign" y "maglinant" se presentan como 1 y 0, respectivamente.

Para empezar a trabajar concatenamos ambos dataframes: características (Features) y variable objetivo (target)

In [ ]:
df = pd.concat([df_features, df_target], axis=1)
df.head()

### Exploración de los datos

Antes de comenzar con la exploración de los datos vamos a asignarle nombre a la variable "target". Por lo tanto, indicaremos que:
- target = 1 (benigno) 
- target = 0 (maligno)

In [ ]:
# Podríamos añadir una nueva columna categórica como hicimos en el análisis de supervivencia
# df.loc[data.target == 1, 'cancer'] = "Malignant"
# df.loc[data.target == 0, 'cancer'] = "Benign"

df['target'] = df['target'].apply(lambda x: "Benign"
                                  if x == 1 else "Malignant")
df.head(5)

In [ ]:
# Obtenemos información adicional sobre el conjunto de datos
df.describe()

#### Distribución de la variable objetivo mediante histograma

In [ ]:
print(df["target"].hist())
print(df['target'].value_counts())

Como ya hemos visto antes, la distribución de la variable objetivo es: 
- 212 - Maligno, 
- 357 - Benigno. 

Para visualizar la distribución de los datos mejor utilizaremos la librería seaborn

In [ ]:
# set_style nos permite cambiar los colores de nuestras gráficas (mirar la documentación)
# sns.set_style('darkgrid')
plt.figure(figsize=(8, 6))
sns.countplot(df['target'])
plt.xlabel("Diagnosis")
plt.title("Distribución de la diagnosis")

### Distribución de características

Ahora echaré un vistazo a la distribución de cada característica y veré en qué se diferencian entre 'benigno' y 'maligno'. Para ver la distribución de múltiples variables, podemos usar el diagrama de violín, el diagrama de enjambre o el diagrama de caja. 
Vamos a probar cada una de estas técnicas.

#### ¡OJO! ESTANDARIZAR LOS DATOS
Para visualizar distribuciones de múltiples características en una figura, primero necesito estandarizar los datos:

In [ ]:
from sklearn.preprocessing import StandardScaler

# Estandarizamos solo las características (features) del dataset
# Si tuviesemos que estandarizar también nuestra variable target se haría por separado
scaler = StandardScaler()
scaler.fit(df_features)
features_scaled = scaler.transform(df_features)

# Concatenamos de nuevo nuestras características estandarizadas con la variable target que queremos predecir
features_scaled = pd.DataFrame(data=features_scaled,
                               columns=df_features.columns)

df_scaled = pd.concat([features_scaled, df['target']], axis=1)

Ahora podemos observar que nuestros datos han sido escalados correctamente

In [ ]:
df_scaled.head(3)

In [ ]:
df_scaled.describe()

Cabe destacar que ahora todas las columnas (features) estan comprendidas en un mismo rango de valoresc

Antes de visualizar vamos a crear un dataframe para que nos sea mas fácil el manejo de los datos para la visualización.

Vamos a utilizar la función pandas.melt(). Esta función es útil para transformar un DataFrame en un formato en el que una o más columnas son variables de identificación (id_vars), mientras que todas las demás columnas, serán consideradas variables de características.

In [ ]:
df_scaled_melt = pd.melt(df_scaled, id_vars='target',
                         var_name='features', value_name='value')
df_scaled_melt.head(3)

Para observar la distribución de los datos vamos a crear tres visualizaciones distintas (aunque con una sería suficiente, el uso de más gráficas de distribución de variables nos puede proporcionar información adicional):
- BOX PLOT:  es un método estandarizado para representar gráficamente una serie de datos numéricos a través de sus cuartiles. De esta manera, se muestran a simple vista la mediana y los cuartiles de los datos, y también pueden representarse sus valores atípicos. 
VEASE (https://es.wikipedia.org/wiki/Diagrama_de_caja) para la interpretación del diagrama de cajas
- SWARM PLOT: Un diagrama de enjambre es otra forma de trazar la distribución de un atributo o la distribución conjunta de un par de atributos.
- VIOLIN PLOT: Los diagramas de violín son similares a los diagramas de caja (box plot), excepto que también muestran la densidad de probabilidad de los datos en diferentes valores. Estos gráficos incluyen un marcador para la mediana de los datos y un cuadro que indica el rango intercuartílico, como en los gráficos de caja estándar. En este diagrama de caja se superpone una estimación de la densidad del núcleo. Al igual que los diagramas de caja, los diagramas de violín se utilizan para representar la comparación de una distribución variable (o distribución de muestra) entre diferentes "categorías".


Como hay 30 características en nuestro dataset utlizaremos 10 para cada uno de los tipos de visualizaciones que hemos expuesto.
 

In [ ]:
def violin_plot(features, name):
    """
    This function creates violin plots of features given in the argument.
    """
    # Create query
    query = ''
    for x in features:
        query += "features == '" + str(x) + "' or "
    query = query[0:-4]

    # Create data for visualization
    data = df_scaled_melt.query(query)

    # Plot figure
    plt.figure(figsize=(12, 6))
    sns.violinplot(x='features',
                   y='value',
                   hue='target',
                   data=data,
                   split=True,
                   inner="quart")
    plt.xticks(rotation=45)
    plt.title(name)
    plt.xlabel("Características")
    plt.ylabel("Datos estandarizados")


def swarm_plot(features, name):
    """
    This function creates swarm plots of features given in the argument.
    """
    # Create query
    query = ''
    for x in features:
        query += "features == '" + str(x) + "' or "
    query = query[0:-4]

    # Create data for visualization
    data = df_scaled_melt.query(query)

    # Plot figure
    plt.figure(figsize=(12, 6))
    sns.swarmplot(x='features', y='value', hue='target', data=data)
    plt.xticks(rotation=45)
    plt.title(name)
    plt.xlabel("Características")
    plt.ylabel("Datos estandarizados")


def box_plot(features, name):
    """
    This function creates box plots of features given in the argument.
    """
    # Create query
    query = ''
    for x in features:
        query += "features == '" + str(x) + "' or "
    query = query[0:-4]

    # Create data for visualization
    data = df_scaled_melt.query(query)

    # Plot figure
    plt.figure(figsize=(12, 6))
    sns.boxplot(x='features', y='value', hue='target', data=data)
    plt.xticks(rotation=45)
    plt.title(name)
    plt.xlabel("Características")
    plt.ylabel("Datos estandarizados")

In [ ]:
# Visualizamos la distribución de las diez primeras características
box_plot(df.columns[0:10], "Box Plot")

In [ ]:
swarm_plot(df.columns[10:20], "Swarm Plot")

In [ ]:
violin_plot(df.columns[20:30], "Violin Plot")

De los gráficos anteriores, podemos extraer algunas ideas de los datos:

- La mediana de algunas características es muy diferente entre "malignas" y "benignas". Esta separación se puede ver claramente en los diagramas de caja. Pueden ser muy buenas características para la clasificación. Por ejemplo:mean radius, mean area, mean concave points, worst radius, worst perimeter, worst area, worst concave points.
- Sin embargo, hay distribuciones que parecen similares entre "maligno" y "benigno". Por ejemplo:  mean smoothness, mean symmetry, mean fractual dimension, smoothness error. Por lo tanto, estas características no nos proporcionarán mucha ayuda en la tarea de clasificación.
- Otras características tienen distribuciones similares, por lo que pueden estar altamente correlacionadas entre sí. Por ejemplo: mean perimeter vs. mean area, mean concavity vs. mean concave points y  worst symmetry vs. worst fractal dimension. Por lo tanto, quizás no deberíamos incluir estas variables altamente correlacionadas.

###  Correlación entre variables

Como se ha observado anteriormente, algunas variables en el conjunto de datos pueden estar altamente correlacionadas entre sí. Exploremos la correlación de los tres ejemplos anteriores.

In [ ]:
def correlation(var):
    """
    1. Print correlation
    2. Create jointplot
    """
    # Print correlation
    print("Correlation: ", df[[var[0], var[1]]].corr().iloc[1, 0])

    # Create jointplot
    plt.figure(figsize=(6, 6))
    sns.jointplot(df[(var[0])], df[(var[1])], kind='reg')

In [ ]:
correlation(['mean perimeter', 'mean area'])

In [ ]:
correlation(['mean concavity', 'mean concave points'])

In [ ]:
correlation(['worst symmetry', 'worst fractal dimension'])

Dos de los tres conjuntos de variables estan altamente correlados. Una correlación mayor al 90%

Para ver si existe mas correlaciones entre variables podemos hacer un estudio genérico de todas las variables mediante un mapa de calor que nos proporcionará la correlación de todas con todas.

In [ ]:
# Creamos la matriz de correlaciones
corr_mat = df.corr()

# Plot heatmap
plt.figure(figsize=(15, 10))
sns.heatmap(corr_mat, annot=True, fmt='.1f',
            cmap='RdBu_r', vmin=-1, vmax=1)

# Podemos crear una máscara para visualizar la mirad del mapa de correlaciones.
# mask = np.zeros_like(corr_mat, dtype=np.bool)
# mask[np.triu_indices_from(mask, k=1)] = True

# # Plot heatmap
# plt.figure(figsize=(15, 10))
# sns.heatmap(corr_mat, annot=True, fmt='.1f',
#             cmap='RdBu_r', vmin=-1, vmax=1,mask=mask)



En el mapa de calor, podemos ver que muchas variables en el conjunto de datos están altamente correlacionadas. 

Podemos poner un valor umbral para ver qué variables tienen una correlación mayor a un 70% por ejemplo

In [ ]:
plt.figure(figsize=(15, 10))
sns.heatmap(corr_mat[corr_mat > 0.5], annot=True, fmt='.1f'
            ,cmap=sns.cubehelix_palette(200))

# Si hemos declarado la mascara podemos visualizarlo con mask
# plt.figure(figsize=(15, 10))
# sns.heatmap(corr_mat[corr_mat > 0.7], annot=True, fmt='.1f'
#             ,cmap=sns.cubehelix_palette(200), mask=mask)

Podemos observar que tenemos muchas variables correlacionadas entre sí. Por lo tanto, podremos aplicar algún algoritmo de selección de variables.

## Paso 2: Preprocesamiento de los datos

Comprobamos si los datos necesitan ser pre-procesados. 

Ahora necesitaríamos estandarizar nuestros datos antes de comenzar a trabajar pero ya lo hemos hecho previamente para visualizar los datos correctamente

In [ ]:
# df.isna().sum()

¡POR SUERTE! 
Los datos han sido limpiados y pre-procesados previamente por lo que podemos saltarnos este paso

### Feature Selection (Selección de variables)

Podemos aplicar un algoritmo de selección de variables para quedarnos con las variables más significativas para nuestra clasificación. 

#### Correlation based feature selection (CFS)
En este caso vamos a utilizar Univariate Feature Selection pero podríamos utilizar/probar otro método de selección de variables. Este es un tipo de selección de características basada en correlación (CFS). Es una técnica de selección de características que utiliza el enfoque de filtro. Esta técnica de selección de características no depende de un algoritmo ML que se aplicará a las características seleccionadas. A menudo, los atributos de las características de un conjunto de datos pueden estar muy correlacionados entre sí. Estas características que se correlacionan en gran medida con otras características brindan información redundante. La técnica encuentra la correlación entre características. Las características que están altamente correlacionadas con otras características son excluido por el CFS. De manera similar, las características que se interrelacionan en gran medida con la etiqueta de la clase se conservan y seleccionan.


https://scikit-learn.org/stable/modules/classes.html?highlight=feature%20select#module-sklearn.feature_selection

Elijo 5 porque en el mapa de calor pude ver alrededor de 5 grupos de características que están altamente correlacionadas. Aunque esto es un factor muy personal.

#### Recursive feature elimination (RFE)
La eliminación de características recursivas (RFE) es un método de selección de características que utiliza el enfoque de envoltura dependiente del modelo que se ha implementado previamente. Con este método de selección de variables primero tendremos que entrenar nuestro modelo con todas las variables para ver la importancia de cada una de las variables sobre el modelo. RFE implica la construcción de un modelo ML con todas las características originales en el conjunto de datos y las características se clasifican de acuerdo con su importancia cuantitativa para reducir el error de modelado.  Cada subconjunto de características se califica con una puntuación de precisión. Los subconjuntos de características que tienen las puntuaciones más altas se eligen.

"Los vínculos entre variables con puntuaciones iguales se romperán de una manera no especificada."

In [ ]:
from sklearn.feature_selection import SelectKBest, chi2
# Definimos feature Selection K=5
feature_selection = SelectKBest(chi2, k=5)
# Fit Feature Selection
feature_selection.fit(df_features, df_target) #Run score function on (X, y) and get the appropriate features.
# Seleccionamos las características
selected_features = df_features.columns[feature_selection.get_support()]
print("Las características selecionadas son: ", list(selected_features))

In [ ]:
feature_selection

In [ ]:
#Get a mask, or integer index, of the features selected.
feature_selection.get_support()

In [ ]:
# Reduce X to the selected features with .transform(X)
X = pd.DataFrame(feature_selection.transform(df_features),
                 columns=selected_features)
X.head()

Vamos a crear un diagrama de pares (PAIRPLOT) para ver cómo se diferencian estas características en 'maligno' y en 'benigno'.

Esta gráfica también es conocida como scatterplot matrix

Una gráfica de pares nos permite ver tanto la distribución de variables individuales como las relaciones entre dos variables. Los diagramas de pares son un gran método para identificar tendencias para el análisis de seguimiento.

El diagrama de pares se basa en dos figuras básicas, el histograma y el diagrama de dispersión. El histograma en la diagonal nos permite ver la distribución de una sola variable, mientras que los diagramas de dispersión en los triángulos superior e inferior muestran la relación (o falta de ella) entre dos variables. 

In [ ]:
# sns.pairplot(pd.concat([X, df['target']], axis=1))
# Le podemos indicar la variable target para visualizar la diferencia que existe entre Maligno y benigno 
# en cada una de las variables y en la relación entre ellas
sns.pairplot(pd.concat([X, df['target']], axis=1), hue='target')

### Ejercicio en clase
Probar otros métodos de selección de características alojados en la librería de Scikit-Learn y comparar los resultados. ¿Son las misma características?

## Paso 3: Preparar los datos

In [ ]:
# Estamos cogiendo los datos una vez aplicada la selección de variables
from sklearn.model_selection import train_test_split
y = df_target['target']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

In [ ]:
X.shape
y.shape

## Paso 4: Selección del modelo

Estamos viendo un caso de uso de clasificación binaria [0:maligno, 1:benigno]. Por lo tanto, debemos de elegir algún algoritmo de clasificación para implementar nuestro modelo. 

Lo primero y más importante para elegir con éxito el algoritmo que más se adapta a nuestras necesidades es:

- Determinar qué queremos conseguir
- Ver qué datos disponemos

Una vez que se tienen claros estos dos puntos y conociendo los algoritmos de machine learning existentes, podremos escoger el que mejor se adapte a nuestras necesidades. Sin embargo, suponiendo que no hemos trabajando nunca en un caso de uso similar probaremos una batería de algoritmos ML y los compararemos para ver cuál de ellos no ofrece mejores resultados en este caso.

Algunos de los algoritmos ML más comunes para clasificación:

- Decision Tree/Random Forest
- K Nearest Neighbor
- Naive Bayes
- Support Vector Machine
- Logistic Regression




### Random Forest Classification

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=200)
rfc.fit(X_train, y_train)
y_pred = rfc.predict(X_test)

## Paso 5:  Evaluación del modelo

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\n")
print("Classification Report:\n", classification_report(y_test, y_pred))

accuracy, recall, f1-score es aproximadamente un  97%. 

Se puede observar en la matriz de confusión que el modelo solo ha fallado en 6 predicciones

### Otro Método para feature selection
Ahora podríamos utilizar todo el conjunto de datos para entrenar nuestro algoritmo de Random Forest y llamar a la función “feature_importances_” para obtener la importancia de variables para este modelo en concreto.

In [ ]:
# Cogemos todos los datos para hacer el SPLIT train/test
from sklearn.model_selection import train_test_split
y = df_target['target']
X_train, X_test, y_train, y_test = train_test_split(
    df_features, y, test_size=0.33, random_state=42)   

In [ ]:
X_train.shape

Primero evaluamos el modelo entrenado con todas las características

In [ ]:
# Fit the model
rfc = RandomForestClassifier(n_estimators=200)
rfc.fit(X_train, y_train)
# Make predictions on test data
y_pred = rfc.predict(X_test)
# Evaluate the model
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\n")
print("Classification Report:\n", classification_report(y_test, y_pred))

Podemos observar que entrenando el modelo con todas las variables obtenemos resultados bastante peores que si aplicamos selección de características. Por este motivo, vamos a probar como funciona otro método de Feature Selection.

In [ ]:
def get_important_features(model):
    feats = {}
    for feature, importance in zip(df_features.columns, model.feature_importances_):
        feats[feature] = importance

    importances = pd.DataFrame.from_dict(feats, orient='index').rename(columns={0: 'Gini-Importance'})
    importances = importances.sort_values(by='Gini-Importance', ascending=False)
    importances = importances.reset_index()
    importances = importances.rename(columns={'index': 'Features'})
#     display(importances)
    return importances

importances = get_important_features(rfc)

def plot_important_features(importances):
    sns.set(font_scale = 5)
    sns.set(style="whitegrid", color_codes=True, font_scale = 1.7)
    fig, ax = plt.subplots()
    fig.set_size_inches(30,15)
    sns.barplot(x=importances['Gini-Importance'], y=importances['Features'], data=importances, color='skyblue')
    plt.xlabel('Importance', fontsize=25, weight = 'bold')
    plt.ylabel('Features', fontsize=25, weight = 'bold')
    plt.title('Feature Importance', fontsize=25, weight = 'bold')
    display(plt.show())

# Plot feature importance
plot_important_features(importances)  

In [ ]:
# Get most_important features
importances = importances.drop(importances[importances['Gini-Importance'] < 0.08].index)
# Plot most_important features
plot_important_features(importances)  

Ahora vamos a entrenar el modelo solo con estas características

In [ ]:
important_features = list(importances['Features'])
print(important_features)

# train_important = df_train[important_features]
# test_important = df_test[important_features]
# # Convert to numpy
# X_train = np.array(train_important)
# X_test = np.array(test_important)

In [ ]:
y = df_target['target']
X = df_features[important_features] #Escogemos solo las columnas mas representativas para el modelo
print(X.head())
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42) 
print(X.shape)

In [ ]:
# Fit the model
rfc = RandomForestClassifier(n_estimators=200)
rfc.fit(X_train, y_train)
# Make predictions on test data
y_pred = rfc.predict(X_test)
# Evaluate the model
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\n")
print("Classification Report:\n", classification_report(y_test, y_pred))

Hemos visto como podemos mejorar la precisión del modelo mediante un pre-procesamiento de los datos con Feature Selection